In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from skimage.io import imread
from skimage.transform import resize
import torch.nn as nn
from torch.optim import Adam
import gc

#constants
grayscale = True
height = 256
width = 256
train = {'id':[],'label':[]}

#maintain folder variables to switch
shirley_folder = '/content/gdrive/My Drive/paths_left_right.csv'
lava_folder = '/content/gdrive/My Drive/Colab Notebooks/ml-for-games-final/paths_left_right_Lava.csv'

#read input file
with open(lava_folder,'r') as f:
    t = f.readlines()

#split data to feature-label
for i in t:
    a,b = i.split("  ")
    lb = b.replace("\n","")
    train['id'].append(a)
    train['label'].append(lb)

#preprocess the image
def preprocess(img):
    img_size = (height, width)
    img = resize(np.array(img), img_size)
    img = np.transpose(img)
    img = img.astype('float32')
    return img

train_img = []
for img_path in tqdm(train['id']):
    img = imread(img_path, as_gray=True)
    train_img.append(preprocess(img))

#defining the train data
train_x = np.array(train_img)
# defining the train label
train_y = np.array(train['label'])

#divide into train and validation set
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1)

#set number of epochs
n_epochs = 50   #44 normal and #5 is lab test
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []

# converting the target into torch format
val_x = val_x.reshape(val_x.shape[0], 1,val_x.shape[1],val_x.shape[2])
val_x  = torch.from_numpy(val_x)

val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)

x_val, y_val = val_x,val_y
y_val = y_val.type(torch.LongTensor)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cnn_layers = nn.Sequential(
        # Defining a 2D convolution layer
        nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        # Defining another 2D convolution layer
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(4096,1024),
            nn.Linear(1024,512),
            nn.Linear(512,2),
        )

    # Defining the forward pass
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=3e-4, weight_decay= 1e-5)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

100%|██████████| 9796/9796 [38:51<00:00,  4.20it/s]


In [4]:

def train(epoch, new_x_train, new_y_train):
    model.train()
    tr_loss = 0
    # converting the data into GPU format
    # clearing the Gradients of the model parameters
    optimizer.zero_grad()

    # prediction for training and validation set
    output_train = model(new_x_train)

    running_corrects_train = 0
    #computing the accuracy
    _, preds_train = torch.max(output_train, 1)
    running_corrects_train += torch.sum(preds_train == new_y_train)

    # computing the training and validation loss
    loss_train = criterion(output_train, new_y_train)
    train_losses.append(loss_train.item())

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()

    model.eval()
    output_val = model(x_val)
    running_corrects_val = 0
    _, preds_test = torch.max(output_val, 1)
    running_corrects_val += torch.sum(preds_test == y_val)
    loss_val = criterion(output_val, y_val)
    val_losses.append(loss_val.item())

    if epoch % 2 == 0:
        # printing the validation loss
        print('Epoch : ', epoch + 1, '\t', 'loss :', loss_val, 'train_acc :', running_corrects_train.double(), 'val_acc :', running_corrects_val.double())

x_val = x_val.to('cuda:0')
y_val = y_val.to('cuda:0')
min_error = float('+inf')
minima_flag = 0
best_model = Net()
tolerance = 40
#set number of epochs
n_epochs = 100   #44 normal and #5 is lab test

# training the model
for epoch in range(n_epochs):
  val_losses = []
  for iteration in range(32):
    indices = np.random.choice(len(train_x),64)
    ipx = torch.from_numpy(train_x[indices].astype(np.float32)).to('cuda:0')
    ipy = torch.from_numpy(train_y[indices].astype(np.long)).to('cuda:0')
    ipx = ipx.view(ipx.shape[0],1,ipx.shape[1],ipx.shape[2])
    train(epoch,ipx,ipy)
    del ipx
    del ipy
    # gc.collect()
    # torch.cuda.empty_cache()
  mean_iteration_error = np.mean(val_losses)
  if mean_iteration_error < min_error:
    min_error = mean_iteration_error
    best_model = model.cpu().state_dict()
    model.cuda()
    #torch.save(model.cpu().state_dict(), PATH)
    minima_flag = 0
  else:
    minima_flag += 1
    if minima_flag > tolerance:
      break

print(str(min_error))


Epoch :  1 	 loss : tensor(0.7078, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(30., device='cuda:0', dtype=torch.float64) val_acc : tensor(496., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.7050, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(31., device='cuda:0', dtype=torch.float64) val_acc : tensor(484., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.7024, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(34., device='cuda:0', dtype=torch.float64) val_acc : tensor(484., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.6933, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(41., device='cuda:0', dtype=torch.float64) val_acc : tensor(480., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.6933, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(38., device='cuda:0', dtype=torch.float64) val_acc : tensor(496., device='cuda:0', dtype=torch

In [5]:
PATH = '/content/gdrive/My Drive/model_left_right_ss.pt'
torch.save(best_model, PATH)